In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

### Board Creation

In [2]:
# Each type of ship and their size
ships = {
    "Carrier" : 5,
    "Battleship" : 4,
    "Cruiser" : 3,
    "Submarine" : 3,
    "Destroyer" : 2
}

total_size = 17
nS = 100
nA = 100

In [3]:
# Randomly places ships on a board
def random_place_ships():
    your_ships = np.zeros((10, 10))
    for ship in ships:
        ship_size = ships[ship]
        right_placements = []
        down_placements = []
        for x in range(10):
            for y in range(10):
                end_d = x + ship_size
                end_r = y + ship_size
                if end_d <= 10 and 2 not in your_ships[:,y][x:end_d]:
                    down_placements.append((x, y))
                if end_r <= 10 and 2 not in your_ships[x][y:end_r]:
                    right_placements.append((x, y))
        if (len(down_placements) > 0 and len(right_placements) > 0):
            d_or_r = random.randint(0, 1)
            if (d_or_r == 1):
                x, y = down_placements[random.randint(0, len(down_placements)- 1)]
                end = x + ship_size
                your_ships[:,y][x:end] = 2
            else:
                x, y = right_placements[random.randint(0, len(right_placements)- 1)]
                end = y + ship_size
                your_ships[x][y:end] = 2
                
    return your_ships

### Q-learning

In [ ]:
# Gets the available shots  
def get_available_shots(ships):
    shots = []
    for i in range(100):
        x, y = reverse_map(i)
        if (ships[x][y] != 1):
            shots.append(i)
    return shots

In [ ]:
# Fires a shot using Q-learning
def fire_shot_q(state, epsilon, Q, ships):
    legal_shots = get_available_shots(ships)
    chance = np.random.rand()
    if chance < epsilon:
        return np.random.choice(legal_shots)
    max_action = legal_shots[0]
    for a in legal_shots:
        if Q[state][a] > Q[state][max_action]:
            max_action = a
    return max_action

In [ ]:
# Returns if a battleship game is over
def over(your_ships, opponent_ships):
    return 2 not in your_ships or 2 not in opponent_ships

In [ ]:
# Reverses a state to a coordinate
def reverse_map(observation):
    return observation//10, observation%10

In [ ]:
# Determiens if a shot hit a ship
# Updates the Reward and ship
def shot_hit(a, s, ships, R):
    x, y = reverse_map(a)
    tile = ships[x][y]
    ships[x][y] = 1
    if (tile == 2):
        R[a][s][a] += 50
        return True
    return False

def q_learning(a, s, ships, R, U, Q):
    shot_hit(a, s, ships, R)
    learning_rate = 1 / (1 + U[s, a])
    Q[s][a] = Q[s][a] + learning_rate * (R[a][s][a] + gamma * np.max(Q[a]) - Q[s][a])
    U[s][a] += 1


In [ ]:
# Creates a deep copy of a board of ships
def copy_placements(ships):
    copy_ships = np.zeros((10, 10))
    for x in range(10):
        for y in range(10):
            copy_ships[x][y] = ships[x][y]
    return copy_ships

In [ ]:
# Runs a round Q-learning training
def run_round_q(s, Q, R, U, e, ships):
    a = fire_shot_q(s, e, Q, ships)
    q_learning(a, s, ships, R, U, Q)
    return a

In [ ]:
# Averages the results of training based on a given interval
def average_results(results, step):
    average_results = []
    for i in range(len(results)):
        if i % step == 0 and i > 0:
            average_results.append(np.mean(results[i - step:i]))
    return average_results

In [ ]:
# Defines control ships
ships1 = random_place_ships()
ships2 = random_place_ships()

#### Running Q-learning (Controlled Simulation)

In [ ]:
accuracy = []
e = 0.9
gamma = 0.9
Q1 = np.zeros((nS, nA))
R1 = np.zeros((nS, nS, nA))
U1 = np.zeros((nS, nA))

Q2 = np.zeros((nS, nA))
R2 = np.zeros((nS, nS, nA))
U2 = np.zeros((nS, nA))

# Training control ships with Q Learning
for i in range(10000):
    s1 = random.randint(0, 99)
    s2 = random.randint(0, 99)
    num_rounds = 0
    your_ships = copy_placements(ships1)
    opponent_ships = copy_placements(ships2)
    while(not over(your_ships, opponent_ships)):
        s1 = run_round_q(s1, Q1, R1, U1, e, your_ships)
        s2 = run_round_q(s2, Q2, R2, U2, e, opponent_ships)
        e *= 0.99999
        num_rounds += 1
    accuracy.append(total_size / num_rounds)

#### Measuring Preformance of Q-learning (Controlled Simulation)

In [ ]:
average_accuracy = average_results(accuracy, 10)

# Plotting average accuracy
plt.plot(average_accuracy)
plt.title("Average Accuracy every 10 episodes")
plt.ylabel("Accuracy")
plt.xlabel("10th Episode")

In [ ]:
# Plotting distribution of accuracy
plt.hist(accuracy, bins=25)
plt.xlabel("Accuracy")
plt.ylabel("Frequency")
plt.title("Accuracy Distribution")

In [ ]:
print("Highest Accuracy: ", np.max(accuracy))
print("Average Accuracy: ", np.mean(accuracy))

#### Running Q-learning (Random Simulation)

In [ ]:
accuracy = []
e = 0.9
gamma = 0.9
Q = np.zeros((nS, nA))
R1 = np.zeros((nS, nS, nA))
U1 = np.zeros((nS, nA))

Q = np.zeros((nS, nA))
R = np.zeros((nS, nS, nA))
U= np.zeros((nS, nA))

# Trains Q Learning with random ships
for i in range(50000):
    s1 = random.randint(0, 99)
    s2 = random.randint(0, 99)
    num_rounds = 0
    your_ships = random_place_ships()
    opponent_ships = random_place_ships()
    while(not over(your_ships, opponent_ships)):
        s1 = run_round_q(s1, Q1, R1, U1, e, your_ships)
        s2 = run_round_q(s2, Q2, R2, U2, e, opponent_ships)
        e *= 0.99999
        num_rounds += 1
    accuracy.append(total_size / num_rounds)

#### Measuring Preformance of Q-learning (Random Simulation)

In [ ]:
average_accuracy = average_results(accuracy, 50)

# Plotting average accuracy
plt.plot(average_accuracy)
plt.title("Average Accuracy every 50 episodes")
plt.ylabel("Accuracy")
plt.xlabel("50th Episode")
plt.title("Accuracy Distribution")

In [ ]:
# Plotting accuracy distribution
plt.hist(accuracy, bins=25)
plt.xlabel("Accuracy")
plt.ylabel("Frequency")
plt.title("Accuracy Distribution")

In [ ]:
print("Highest Accuracy: ", np.max(accuracy))
print("Average Accuracy: ", np.mean(accuracy))

### SARSA

In [ ]:
# Returns if the AI has located a ship
def found_ship(prev_hits):
    n = len(prev_hits)
    if n > 1:
        shot_difference = abs(prev_hits[n - 1] - prev_hits[n - 2])
        return shot_difference == 1 or shot_difference == 10
    return False

# Shoots at a found ships
def hit_ships(state, prev_hits, legal_shots):
    n = len(prev_hits)
    was_hit = False
    if (n > 0):
        was_hit = prev_hits[n- 1] == state
    if (was_hit):
        if (found_ship(prev_hits) and state + (prev_hits[n - 1] - prev_hits[n - 2]) in legal_shots):
            return state + (prev_hits[n - 1] - prev_hits[n - 2]) 
    return None

# Fires a shot using SARSA
def fire_shot_sarsa(state, e, Q, ships, prev_hits):
    legal_shots = get_available_shots(ships)
    if (len(legal_shots) == 0):
        return None
    ship_hit = hit_ships(state, prev_hits, legal_shots)
    if (ship_hit):
        return ship_hit
    return fire_shot_q(state, e, Q, ships)

In [ ]:
# Updates SARSA model
def sarsa(s, a, a_next, ships, R, U, Q):
    learning_rate = 1 / (1 + U[s, a])
    Q[s][a] = Q[s][a] + learning_rate * (R[a][s][a] + gamma * Q[a][a_next] - Q[s][a])
    U[s][a] += 1

In [ ]:
# Runs a round of SARSA training
def run_round_sarsa(s, e, a, ships, Q, R, U, prev_hits):
    hit = 0
    if (shot_hit(a, s, ships, R)):
        hit = 1
        prev_hits.append(a)
    a_next = fire_shot_sarsa(a, e, Q, ships, prev_hits)
    if (a_next == None):
        return None
    sarsa(s, a, a_next, ships, R, U, Q)
    return a_next

#### Running SARSA (Controlled Simulation)

In [ ]:
accuracy = []
gamma = 0.9
e = 0.9
Q1 = np.zeros((nS, nA))
R1 = np.zeros((nS, nS, nA))
U1 = np.zeros((nS, nA))

Q2 = np.zeros((nS, nA))
R2 = np.zeros((nS, nS, nA))
U2 = np.zeros((nS, nA))

# Trains control ships with SARSA
for i in range(10000):
    your_ships = copy_placements(ships1)
    opponent_ships = copy_placements(ships2)
    s1 = random.randint(0, 99)
    s2 = random.randint(0, 99)
    prev_hits1 = []
    prev_hits2 = []
    a1 = fire_shot_sarsa(s1, e, Q1, your_ships, prev_hits1)
    a2 = fire_shot_sarsa(s2, e, Q2, opponent_ships, prev_hits2)
    num_rounds = 0
    while(not over(your_ships, opponent_ships)):
        a1_next = run_round_sarsa(s1, e, a1, your_ships, Q1, R1, U1, prev_hits1)
        if (a1_next == None):
            break
        s1 = a1
        a1 = a1_next
        a2_next = run_round_sarsa(s2, e, a2, opponent_ships, Q2, R2, U2, prev_hits2)
        if (a2_next == None):
            break
        s2 = a2
        a2_next = a2
        num_rounds += 1
        e *= 0.99999
    accuracy.append(total_size / num_rounds)

#### Measuring Preformance SARSA (Controlled Simulation)

In [ ]:
average_accuracy = average_results(accuracy, 10)

# Plotting average accuracy
plt.plot(average_accuracy)
plt.title("Average Accuracy every 10 episodes")
plt.ylabel("Accuracy")
plt.xlabel("10th Episode")

In [ ]:
# Plotting accuracy distribution
plt.hist(accuracy, bins=25)
plt.xlabel("Accuracy")
plt.ylabel("Frequency")
plt.title("Accuracy Distribution")

In [ ]:
print("Highest Accuracy: ", np.max(accuracy))
print("Average Accuracy: ", np.mean(accuracy))

#### Running SARSA (Random Simulation)

In [ ]:
accuracy = []
gamma = 0.9
e = 0.9
Q1 = np.zeros((nS, nA))
R1 = np.zeros((nS, nS, nA))
U1 = np.zeros((nS, nA))

Q2 = np.zeros((nS, nA))
R2 = np.zeros((nS, nS, nA))
U2 = np.zeros((nS, nA))

# Trains random ships with SARSA
for i in range(50000):
    your_ships = random_place_ships()
    opponent_ships = random_place_ships()
    s1 = random.randint(0, 99)
    s2 = random.randint(0, 99)
    prev_hits1 = []
    prev_hits2 = []
    a1 = fire_shot_sarsa(s1, e, Q1, your_ships, prev_hits1)
    a2 = fire_shot_sarsa(s2, e, Q2, opponent_ships, prev_hits2)
    num_rounds = 0
    while(not over(your_ships, opponent_ships)):
        a1_next = run_round_sarsa(s1, e, a1, your_ships, Q1, R1, U1, prev_hits1)
        if (a1_next == None):
            break
        s1 = a1
        a1 = a1_next
        a2_next = run_round_sarsa(s2, e, a2, opponent_ships, Q2, R2, U2, prev_hits2)
        if (a2_next == None):
            break
        s2 = a2
        a2_next = a2
        num_rounds += 1
        e *= 0.99999
    accuracy.append(total_size / num_rounds)

#### Measuring Preformance SARSA (Random Simulation)

In [ ]:
average_accuracy = average_results(accuracy, 50)

# Plotting average accuracy
plt.plot(average_accuracy)
plt.title("Average Accuracy every 50 episodes")
plt.ylabel("Accuracy")
plt.xlabel("50th Episode")


In [ ]:
# Plotting accuracy distribution
plt.hist(accuracy, bins=25)
plt.xlabel("Accuracy")
plt.ylabel("Frequency")
plt.title("Accuracy Distribution")

In [ ]:
print("Highest Accuracy: ", np.max(accuracy))
print("Average Accuracy: ", np.mean(accuracy))